## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-21-05-49-16 +0000,nypost,"Providence mayor, police chief honored at coll...",https://nypost.com/2025/12/21/us-news/providen...
1,2025-12-21-05-36-47 +0000,nyt,"Long Before Bondi Massacre, Australian Jews Li...",https://www.nytimes.com/2025/12/21/world/austr...
2,2025-12-21-05-33-41 +0000,nypost,Powerball jackpot jumps up to $1.6B — fourth-l...,https://nypost.com/2025/12/21/us-news/powerbal...
3,2025-12-21-05-28-51 +0000,startribune,Vikings’ charter flight to Newark returns to M...,https://www.startribune.com/vikings-team-plane...
4,2025-12-21-05-28-26 +0000,startribune,Ejection reveals a side of Timberwolves’ Chris...,https://www.startribune.com/ejection-reveals-a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2430/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
122,trump,18
120,epstein,13
121,files,10
186,venezuela,8
13,bondi,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
28,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...,53
46,2025-12-20-23-45-01 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...,53
63,2025-12-20-21-46-48 +0000,nypost,Russian envoy KO’s Trump administration propos...,https://nypost.com/2025/12/20/world-news/russi...,48
71,2025-12-20-20-40-00 +0000,wsj,Donald Trump is finding the Epstein files are ...,https://www.wsj.com/politics/policy/for-trump-...,45
107,2025-12-20-15-32-25 +0000,nypost,Israel’s Benjamin Netanyahu expected to pitch ...,https://nypost.com/2025/12/20/world-news/israe...,44


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
28,53,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...
76,43,2025-12-20-20-09-15 +0000,nypost,Sick new details about Jeffrey Epstein’s crime...,https://nypost.com/2025/12/20/us-news/sick-new...
63,30,2025-12-20-21-46-48 +0000,nypost,Russian envoy KO’s Trump administration propos...,https://nypost.com/2025/12/20/world-news/russi...
87,29,2025-12-20-18-21-29 +0000,nypost,US kills ‘at least 5’ ISIS thugs in Syria in r...,https://nypost.com/2025/12/20/us-news/us-shock...
98,28,2025-12-20-16-36-01 +0000,nypost,Former Hamas hostage warned leaders of rising ...,https://nypost.com/2025/12/20/world-news/forme...
159,28,2025-12-20-09-00-00 +0000,cbc,Carney lays out security 'guardrails' for Chin...,https://www.cbc.ca/news/politics/canada-china-...
107,26,2025-12-20-15-32-25 +0000,nypost,Israel’s Benjamin Netanyahu expected to pitch ...,https://nypost.com/2025/12/20/world-news/israe...
106,24,2025-12-20-15-48-19 +0000,nypost,7 elephants killed after high-speed train coll...,https://nypost.com/2025/12/20/world-news/high-...
80,22,2025-12-20-19-38-33 +0000,latimes,"Ronald Reagan biographer, legendary California...",https://www.latimes.com/california/story/2025-...
0,21,2025-12-21-05-49-16 +0000,nypost,"Providence mayor, police chief honored at coll...",https://nypost.com/2025/12/21/us-news/providen...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
